# Testing integration with `vegas`

__Author:__ A. J. Tropiano [atropiano@anl.gov]<br/>
__Date:__ October 11, 2022

Trying to do a spectroscopic overlap calculation evaluating the integrals in the $\delta U^\dagger$ term with Monte Carlo integration using `vegas`. Currently attempting to do all the sums in the expression by looping (excluding as many terms equal to zero as possible), and use `vegas` only for integration over momenta.

_Last update:_ June 1, 2023

### Install `vegas`

In [1]:
# %%bash
# pip install vegas

In [2]:
# Python imports
import functools
# from numba import njit, vectorize, guvectorize, float64
import numpy as np
import numpy.linalg as la
import pandas as pd
from scipy.interpolate import interp1d, RectBivariateSpline
from scipy.special import spherical_jn, sph_harm
import shutil
from sympy.physics.quantum.cg import CG
import time
import vegas

In [3]:
# Imports from scripts
# ...

## Example from tutorial [online](https://vegas.readthedocs.io/en/latest/tutorial.html)

In [4]:
def f(x):
    dx2 = 0
    for d in range(4):
        dx2 += (x[d] - 0.5) ** 2
    return np.exp(-dx2 * 100.) * 1013.2118364296088

In [5]:
integ = vegas.Integrator([[-1, 1], [0, 1], [0, 1], [0, 1]])
integ(f, nitn=10, neval=1e3)
%time result = integ(f, nitn=10, neval=1e4)
print(result.summary())
print('result = %s    Q = %.2f' % (result, result.Q))

CPU times: user 180 ms, sys: 861 µs, total: 181 ms
Wall time: 181 ms
itn   integral        wgt average     chi2/dof        Q
-------------------------------------------------------
  1   1.0018(32)      1.0018(32)          0.00     1.00
  2   1.0016(23)      1.0017(19)          0.00     0.95
  3   0.9998(20)      1.0008(14)          0.23     0.79
  4   1.0030(17)      1.0017(11)          0.49     0.69
  5   0.9996(16)      1.00106(88)         0.67     0.61
  6   0.9970(16)      1.00008(77)         1.58     0.16
  7   0.9992(13)      0.99987(66)         1.37     0.22
  8   1.0030(11)      1.00066(57)         1.97     0.06
  9   0.9969(13)      1.00004(52)         2.61     0.01
 10   0.9993(12)      0.99992(48)         2.35     0.01

result = 0.99992(48)    Q = 0.01


## Integrand depends on other parameters

Here we use `functools.partial` to input parameters into the integrand function.
This will be useful for practical purposes where the integrand depends on things other than integration variables.

In [6]:
def g(y, x):
    dx2 = 0
    for d in range(4):
        dx2 += (x[d] - y) ** 2
    return np.exp(-dx2 * 100.) * 1013.2118364296088

In [7]:
# Results here should be the same!
x = np.array((0.1, 0.2, 0.3, 0.1))
print(f(x), g(0.5, x))

2.900337707812307e-17 2.900337707812307e-17


In [8]:
integ = vegas.Integrator([[-1, 1], [0, 1], [0, 1], [0, 1]])
y = 0.5
integrand = functools.partial(g, y)
integ(integrand, nitn=10, neval=1e3)
%time result = integ(integrand, nitn=10, neval=1e4)
print(result.summary())
print('result = %s    Q = %.2f' % (result, result.Q))

CPU times: user 182 ms, sys: 648 µs, total: 183 ms
Wall time: 183 ms
itn   integral        wgt average     chi2/dof        Q
-------------------------------------------------------
  1   0.9960(35)      0.9960(35)          0.00     1.00
  2   1.0006(24)      0.9991(20)          1.19     0.27
  3   1.0000(20)      0.9996(14)          0.66     0.52
  4   0.9990(17)      0.9993(11)          0.46     0.71
  5   0.9985(16)      0.99907(90)         0.39     0.82
  6   1.0006(14)      0.99952(76)         0.48     0.79
  7   1.0002(13)      0.99968(66)         0.43     0.86
  8   0.9998(12)      0.99972(57)         0.37     0.92
  9   1.0014(11)      1.00008(51)         0.55     0.82
 10   0.9979(12)      0.99976(47)         0.79     0.63

result = 0.99976(47)    Q = 0.63


## Batch integrand

Function `f_batch(x)` accepts an array of integration points `x[i, d]` where `i=0...` labels the integration point and `d=0...` the direction and returns an array of integrand values corresponding to those points.
The decorator `vegas.batchintegrand()` tells `vegas` that it should send integration points to `f(x)` in batches.

In [9]:
def f(x):
    """Example integrand which returns a float given an integration point x with
    shape (4,).
    """
    dim = len(x)
    norm = 1013.2118364296088 ** (dim / 4.)
    dx2 = 0.0
    for d in range(dim):
        dx2 += (x[d] - 0.5) ** 2
    return np.exp(-100. * dx2) * norm

# integ = vegas.Integrator(4 * [[0, 1]])
integ = vegas.Integrator([[0,1],[0,1],[0,1],[0,1]])

integ(f, nitn=10, neval=2e5)
%time result = integ(f, nitn=10, neval=2e5)
print(result.summary())
print('result = %s    Q = %.2f' % (result, result.Q))

CPU times: user 3.69 s, sys: 15.3 ms, total: 3.71 s
Wall time: 3.71 s
itn   integral        wgt average     chi2/dof        Q
-------------------------------------------------------
  1   0.99978(36)     0.99978(36)         0.00     1.00
  2   0.99873(32)     0.99921(24)         4.80     0.03
  3   1.00057(29)     0.99976(18)         9.00     0.00
  4   0.99991(26)     0.99981(15)         6.07     0.00
  5   1.00001(24)     0.99986(13)         4.68     0.00
  6   1.00036(22)     0.99999(11)         4.47     0.00
  7   0.99990(21)     0.999967(98)        3.75     0.00
  8   0.99985(20)     0.999945(89)        3.26     0.00
  9   1.00016(19)     0.999982(81)        2.98     0.00
 10   0.99983(19)     0.999959(74)        2.71     0.00

result = 0.999959(74)    Q = 0.00


In [10]:
@vegas.batchintegrand
def f_batch(x):
    """Example integrand which evaluates multiple points simultaneously. Here x
    is several integration points with shape (N,4), where N is the number of
    integration points and 4 is the dimension of the integral (meaning there are
    four integration varibles).
    """
    # evaluate integrand at multiple points simultaneously
    dim = x.shape[1]
    norm = 1013.2118364296088 ** (dim / 4.)
    dx2 = 0.0
    for d in range(dim):
        dx2 += (x[:, d] - 0.5) ** 2
    return np.exp(-100. * dx2) * norm

integ = vegas.Integrator(4 * [[0, 1]])

integ(f_batch, nitn=10, neval=2e5)
%time result = integ(f_batch, nitn=10, neval=2e5)
print(result.summary())
print('result = %s    Q = %.2f' % (result, result.Q))

CPU times: user 159 ms, sys: 5.31 ms, total: 164 ms
Wall time: 164 ms
itn   integral        wgt average     chi2/dof        Q
-------------------------------------------------------
  1   1.00002(36)     1.00002(36)         0.00     1.00
  2   0.99975(32)     0.99987(24)         0.31     0.58
  3   0.99977(29)     0.99983(18)         0.19     0.83
  4   1.00042(26)     1.00003(15)         1.26     0.29
  5   0.99993(24)     1.00000(13)         0.97     0.42
  6   0.99987(22)     0.99997(11)         0.83     0.53
  7   1.00004(21)     0.999981(98)        0.71     0.65
  8   0.99988(20)     0.999962(88)        0.63     0.73
  9   0.99977(20)     0.999930(81)        0.65     0.73
 10   0.99974(20)     0.999902(75)        0.67     0.73

result = 0.999902(75)    Q = 0.73


In [11]:
@vegas.batchintegrand
class f_batch_class:
    def __init__(self, dim, y):
        self.dim = dim
        self.norm = 1013.2118364296088 ** (dim / 4.)
        self.y = y

    def __call__(self, x):
        # evaluate integrand at multiple points simultaneously
        dx2 = 0.0
        for d in range(self.dim):
            dx2 += (x[:, d] - self.y) ** 2
        return np.exp(-100. * dx2) * self.norm

f = f_batch_class(dim=4, y=0.5)
integ = vegas.Integrator(f.dim * [[0, 1]])

integ(f, nitn=10, neval=2e5)
%time result = integ(f, nitn=10, neval=2e5)
print(result.summary())
print('result = %s    Q = %.2f' % (result, result.Q))

CPU times: user 159 ms, sys: 3.73 ms, total: 162 ms
Wall time: 162 ms
itn   integral        wgt average     chi2/dof        Q
-------------------------------------------------------
  1   0.99994(36)     0.99994(36)         0.00     1.00
  2   0.99984(32)     0.99989(24)         0.05     0.82
  3   0.99973(29)     0.99982(19)         0.11     0.90
  4   1.00022(27)     0.99995(15)         0.56     0.64
  5   0.99995(24)     0.99995(13)         0.42     0.79
  6   1.00001(22)     0.99997(11)         0.35     0.88
  7   1.00002(21)     0.999979(99)        0.30     0.94
  8   0.99984(20)     0.999952(89)        0.31     0.95
  9   0.99988(20)     0.999940(81)        0.28     0.97
 10   0.99993(19)     0.999939(75)        0.25     0.99

result = 0.999939(75)    Q = 0.99


## Sums with `vegas`

$$
\sum_{i=0}^{N-1} f(i) = N \int_0^1 dx f(\mathrm{floor}(xN))
$$

In [12]:
def ridge(x):
    """Integrand: ridge of N Gaussians spread evenly along the diagonal"""
    N = 10000
    dim = 4
    x0 = 0.4 + 0.2 * np.floor(x[-1] * N) / (N - 1.)
    dx2 = 0.0
    for xd in x[:-1]:
        dx2 += (xd - x0) ** 2
    return np.exp(-100. * dx2) *  (100. / np.pi) ** (dim / 2.)

In [13]:
def main():
    integ = vegas.Integrator(5 * [[0, 1]])
    # adapt
    integ(ridge, nitn=10, neval=5e4)
    # final results
    result = integ(ridge, nitn=10, neval=5e4)
    print('result = %s    Q = %.2f' % (result, result.Q))

In [14]:
main()

result = 1.00057(83)    Q = 0.55


## Integrand depends on vector parameter

In [79]:
def ridge2(y_vector, x):
    """Integrand: ridge of N Gaussians spread evenly along the diagonal"""
    N = 10000
    dim = 4
    x0 = 0.4 + 0.2 * np.floor(x[-1] * N) / (N - 1.)
    dx2 = 0.0
    for xd in x[:-1]:
        dx2 += (xd - x0) ** 2
    return np.exp(-y_vector * dx2) *  (100. / np.pi) ** (dim / 2.)

In [80]:
def main2(y_vector):
    integ = vegas.Integrator(5 * [[0, 1]])
    
    integrand = functools.partial(ridge2, y_vector)
    # adapt
    integ(integrand, nitn=5, neval=5e4)
    # final results
    result = integ(integrand, nitn=10, neval=5e4)
    print('result = %s    Q = %.2f' % (result, result.Q))
    return result

In [81]:
y_vector = np.array([100, 102, 104, 106, 98])
r = main2(y_vector)

result = [1.00029(92) 0.96140(90) 0.92473(88) 0.89011(87) 1.04158(94)]    Q = 0.00


In [83]:
print(r.summary(extended=True), r[0].mean, r[0].sdev)

itn   integral        wgt average     chi2/dof        Q
-------------------------------------------------------
  1   1.0052(38)      1.0078(38)          0.00     1.00
  2   0.9958(34)      1.0007(25)           nan      nan
  3   1.0029(32)      1.0019(20)           nan      nan
  4   1.0025(30)      1.0009(16)         12.18     0.00
  5   1.0068(29)      1.0042(14)           nan      nan
  6   0.9987(28)      1.0005(13)         42.21     0.00
  7   1.0022(28)      1.0026(12)           nan      nan
  8   0.9970(27)      1.0019(11)           nan      nan
  9   1.0012(27)      1.00377(99)          nan      nan
 10   0.9987(26)      1.00029(92)        14.96     0.00

  index           value
-----------------------
      0    1.00029 (92)
      1    0.96140 (90)
      2    0.92473 (88)
      3    0.89011 (87)
      4    1.04158 (94)
 1.0002870338637397 0.0009215565657804109
